# setup

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  0.23.1
TF version:  2.3.1
GPU installed:  True
2 Physical GPUs, 2 Logical GPUs


# Get the data

In [7]:
import tensorflow_datasets as tfds
# You can find lists of datasets at https://www.tensorflow.org/datasets/catalog/overview
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
# Information of dataset split (train/validation/test)
info.splits
# Information of train dataset
info.splits["train"]
# Name of each class
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes
print(n_classes)
dataset_size = info.splits["train"].num_examples
# Split dataset using split method
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)
from functools import partial

# Define preprocess functions
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

# Select central crop or randomized crop
def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.mobilenet.preprocess_input(resized_image)
    return final_image, label

# Define Train, validation and test datasets
batch_size = 16
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

5


In [4]:
# Load Fashion MNIST dataset and split it to train, valid and test data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]
# Normalizing data
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
n_classes=10
print(X_train.shape, y_train.shape)

(55000, 28, 28, 1) (55000,)


In [5]:
def preprocessing_smv2(X_train,y_train):
    for i in range(10):
        temp=lambda :tf.constant(tf.image.resize(X_train[(i)*100:(i+1)*100],[224,224]))
        yield temp,y_train[(i)*100:(i+1)*100]

# SMV2L

In [24]:
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [19]:
smv2_l=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[224,224,3]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])

In [10]:
smv2_l.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 112, 112, 16)      1216      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 56, 56, 16)        6688      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 28, 28, 32)        38656     
_________________________________________________________________
IRB4 (Inveted_Residual_Block (None, 14, 14, 64)        134656    
_________________________________________________________________
IRB5 (Inveted_Residual_Block (None, 7, 7, 128)         2

In [6]:
smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

In [8]:
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam2.h5",save_best_only=True)
history=smv2_l.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10,callbacks=[check_cb])

Epoch 1/10
172/172 [==============================] - 17s 96ms/step - loss: 1.5512 - accuracy: 0.3779 - val_loss: 1.7621 - val_accuracy: 0.1654
Epoch 2/10
172/172 [==============================] - 17s 96ms/step - loss: 1.3904 - accuracy: 0.4415 - val_loss: 1.7141 - val_accuracy: 0.2390
Epoch 3/10
172/172 [==============================] - 16s 92ms/step - loss: 1.2982 - accuracy: 0.4964 - val_loss: 2.8134 - val_accuracy: 0.2390
Epoch 4/10
172/172 [==============================] - 16s 93ms/step - loss: 1.2171 - accuracy: 0.5302 - val_loss: 1.2407 - val_accuracy: 0.5092
Epoch 5/10
172/172 [==============================] - 16s 96ms/step - loss: 1.1591 - accuracy: 0.5425 - val_loss: 1.2270 - val_accuracy: 0.5772
Epoch 6/10
172/172 [==============================] - 16s 93ms/step - loss: 1.0863 - accuracy: 0.5879 - val_loss: 1.1859 - val_accuracy: 0.5423
Epoch 7/10
172/172 [==============================] - 16s 96ms/step - loss: 1.0530 - accuracy: 0.5981 - val_loss: 0.9143 - val_accuracy:

# SMV2P

In [32]:
smv2_p=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[224,224,3]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=128,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])

In [33]:
smv2_p.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_304 (Bat (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_201 (Conv2D)          (None, 112, 112, 32)      896       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 112, 112, 16)      1216      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 56, 56, 16)        6688      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 28, 28, 64)        126400    
_________________________________________________________________
IRB4 (Inveted_Residual_Block (None, 14, 14, 128)       498688    
_________________________________________________________________
IRB5 (Inveted_Residual_Block (None, 7, 7, 128)        

In [34]:
smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [35]:
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam.h5",save_best_only=True)
history=smv2_p.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=300,callbacks=[check_cb])

Epoch 1/300
  2/172 [..............................] - ETA: 13s - loss: 5.0833 - accuracy: 0.2188WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0263s vs `on_train_batch_end` time: 0.0631s). Check your callbacks.


172/172 [==============================] - 17s 98ms/step - loss: 1.9443 - accuracy: 0.3550 - val_loss: 1.9443 - val_accuracy: 0.2390
Epoch 2/300
172/172 [==============================] - 16s 93ms/step - loss: 1.3599 - accuracy: 0.4568 - val_loss: 1.7358 - val_accuracy: 0.2390
Epoch 3/300
172/172 [==============================] - 16s 92ms/step - loss: 1.2830 - accuracy: 0.5025 - val_loss: 2.4508 - val_accuracy: 0.2390
Epoch 4/300
172/172 [==============================] - 16s 92ms/step - loss: 1.2118 - accuracy: 0.5247 - val_loss: 2.8804 - val_accuracy: 0.2831
Epoch 5/300
172/172 [==============================] - 17s 97ms/step - loss: 1.1444 - accuracy: 0.5552 - val_loss: 1.1505 - val_accuracy: 0.5882
Epoch 6/300
172/172 [==============================] - 16s 91ms/step - loss: 1.0970 - accuracy: 0.5799 - val_loss: 1.2076 - val_accuracy: 0.5882
Epoch 7/300
172/172 [==============================] - 16s 94ms/step - loss: 1.0259 - accuracy: 0.6072 - val_loss: 1.0767 - val_accuracy: 0.62

172/172 [==============================] - 17s 96ms/step - loss: 0.3884 - accuracy: 0.8616 - val_loss: 0.7683 - val_accuracy: 0.7721
Epoch 58/300
172/172 [==============================] - 17s 96ms/step - loss: 0.4141 - accuracy: 0.8434 - val_loss: 0.5866 - val_accuracy: 0.7886
Epoch 59/300
172/172 [==============================] - 17s 96ms/step - loss: 0.3941 - accuracy: 0.8517 - val_loss: 0.8264 - val_accuracy: 0.7647
Epoch 60/300
172/172 [==============================] - 17s 97ms/step - loss: 0.3795 - accuracy: 0.8688 - val_loss: 0.7539 - val_accuracy: 0.7371
Epoch 61/300
172/172 [==============================] - 17s 96ms/step - loss: 0.3894 - accuracy: 0.8594 - val_loss: 0.5924 - val_accuracy: 0.8254
Epoch 62/300
172/172 [==============================] - 17s 96ms/step - loss: 0.4004 - accuracy: 0.8586 - val_loss: 0.6451 - val_accuracy: 0.7757
Epoch 63/300
172/172 [==============================] - 17s 96ms/step - loss: 0.3963 - accuracy: 0.8590 - val_loss: 0.6711 - val_accuracy

172/172 [==============================] - 16s 96ms/step - loss: 0.2520 - accuracy: 0.9088 - val_loss: 0.5643 - val_accuracy: 0.8217
Epoch 114/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2398 - accuracy: 0.9182 - val_loss: 0.6026 - val_accuracy: 0.8143
Epoch 115/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2427 - accuracy: 0.9135 - val_loss: 0.6663 - val_accuracy: 0.8051
Epoch 116/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2361 - accuracy: 0.9157 - val_loss: 0.7070 - val_accuracy: 0.8199
Epoch 117/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2287 - accuracy: 0.9146 - val_loss: 0.6729 - val_accuracy: 0.7923
Epoch 118/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2385 - accuracy: 0.9088 - val_loss: 0.6157 - val_accuracy: 0.8199
Epoch 119/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2634 - accuracy: 0.9070 - val_loss: 0.5803 - val_ac

Epoch 169/300
172/172 [==============================] - 17s 97ms/step - loss: 0.2216 - accuracy: 0.9219 - val_loss: 0.7463 - val_accuracy: 0.8015
Epoch 170/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1906 - accuracy: 0.9331 - val_loss: 0.7281 - val_accuracy: 0.8051
Epoch 171/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2017 - accuracy: 0.9248 - val_loss: 0.7066 - val_accuracy: 0.8272
Epoch 172/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2003 - accuracy: 0.9270 - val_loss: 0.5965 - val_accuracy: 0.8217
Epoch 173/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1816 - accuracy: 0.9368 - val_loss: 0.6860 - val_accuracy: 0.8235
Epoch 174/300
172/172 [==============================] - 17s 97ms/step - loss: 0.2034 - accuracy: 0.9320 - val_loss: 0.6723 - val_accuracy: 0.8143
Epoch 175/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2000 - accuracy: 0.9299 - val_loss: 0

Epoch 225/300
172/172 [==============================] - 17s 97ms/step - loss: 0.1844 - accuracy: 0.9310 - val_loss: 0.7037 - val_accuracy: 0.7978
Epoch 226/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1924 - accuracy: 0.9302 - val_loss: 0.6532 - val_accuracy: 0.8254
Epoch 227/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1822 - accuracy: 0.9331 - val_loss: 0.8370 - val_accuracy: 0.7868
Epoch 228/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2176 - accuracy: 0.9251 - val_loss: 0.7560 - val_accuracy: 0.7960
Epoch 229/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1866 - accuracy: 0.9291 - val_loss: 0.7470 - val_accuracy: 0.8107
Epoch 230/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1934 - accuracy: 0.9339 - val_loss: 0.7315 - val_accuracy: 0.8015
Epoch 231/300
172/172 [==============================] - 17s 96ms/step - loss: 0.2080 - accuracy: 0.9295 - val_loss: 0

Epoch 281/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1764 - accuracy: 0.9411 - val_loss: 0.6291 - val_accuracy: 0.8162
Epoch 282/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1559 - accuracy: 0.9484 - val_loss: 0.7095 - val_accuracy: 0.8143
Epoch 283/300
172/172 [==============================] - 17s 97ms/step - loss: 0.1618 - accuracy: 0.9400 - val_loss: 0.7685 - val_accuracy: 0.8125
Epoch 284/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1837 - accuracy: 0.9328 - val_loss: 0.6538 - val_accuracy: 0.8235
Epoch 285/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1534 - accuracy: 0.9433 - val_loss: 0.6417 - val_accuracy: 0.8125
Epoch 286/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1856 - accuracy: 0.9339 - val_loss: 0.7332 - val_accuracy: 0.7886
Epoch 287/300
172/172 [==============================] - 17s 96ms/step - loss: 0.1983 - accuracy: 0.9302 - val_loss: 0

In [31]:
model=keras.models.load_model("smv2l_nadam.h5",custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})
model.evaluate(test_set)

23/23 [==============================] - 0s 18ms/step - loss: 0.4342 - accuracy: 0.2918


[0.4341720938682556, 0.29178759455680847]

In [30]:
model2=keras.models.load_model("my_mobilenetv2_1.h5",
                               custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})
model2.evaluate(test_set)

23/23 [==============================] - 1s 23ms/step - loss: 1.1099 - accuracy: 0.3706


[1.1099331378936768, 0.37057220935821533]

In [36]:
model3=keras.models.load_model("smv2p_nadam.h5",
                               custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})
model3.evaluate(test_set)

23/23 [==============================] - 0s 19ms/step - loss: 0.5485 - accuracy: 0.2779


[0.5485105514526367, 0.2779291570186615]

In [37]:
model.evaluate(valid_set)

35/35 [==============================] - 1s 19ms/step - loss: 0.4999 - accuracy: 0.2585


[0.4998670816421509, 0.25853991508483887]

In [38]:
model3.evaluate(valid_set)

35/35 [==============================] - 1s 24ms/step - loss: 0.5297 - accuracy: 0.2468


[0.5296900868415833, 0.24682395160198212]